In [76]:
from pathlib import Path
import json
import os

from tinydb import TinyDB,Query

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)


class DocumentDB(object):
    def __init__(self, db_path):
        
        #//*** Set Local Path variable
        self._db_path = db_path
        
        #//**** JSON file Paths
        people_json = kv_data_dir.joinpath('people.json')
        visited_json = kv_data_dir.joinpath('visited.json')
        sites_json = kv_data_dir.joinpath('sites.json')
        measurements_json = kv_data_dir.joinpath('measurements.json')
      
        #//*** Load JSON content from each json filepath
        people = self._read_file(people_json)
        visited = self._read_file(visited_json)
        sites = self._read_file(sites_json)
        measurements = self._read_file(measurements_json)
        
        
        #//*****************************************************************************************************
        #//*** Combine All the data in a dictionary, then import to TinyDB. Not sure if this is the proper way,
        #//*** But the source and output requirements are a bit disjointed. We'll Assemble a dictionry
        #//*** Then feed the properly formatted dictionary to TinyDB.
        #//*****************************************************************************************************
        #//*** All output is tied to people. Using that as a base dictionary
        #//*** person_id and visit_id are linked in the measurements keys
        #//*****************************************************************************************************
        
        #//*** Get Visit IDs from Measurements
        for visit_id in measurements.keys():
            
            #//*** Get the PersonIDs from each visit
            for person_id in measurements[visit_id].keys():
                
                #//*** Add Visits list to the dictionary if it doesn't exist
                if "visits" not in people[person_id].keys():
                    people[person_id]["visits"] = []
                
                #//*** Grab the site_id. For simiplicity sake, there is only one
                #//*** One site_id in the keys, otherwise I'd have to loop this
                site_id = list(visited[visit_id].keys())[0]
                
                #//************************************************************
                #//*** Loop_visit is a temporary dictionary to hold the site 
                #//*** visit values for each loop
                #//************************************************************
                
                #//*** Get the visit values based on the visit_id and site_id
                loop_visit = visited[visit_id][site_id]
                
                #//*** Add Site Key based on site_id in sites.json
                loop_visit['site'] = sites[site_id]
                
                #//*** Initialize Measurements as a list
                loop_visit['measurements'] = []
                
                #//*** Measurement values are the third JSON key (or third tuple value in kvdb.ipynb)
                #//*** Append each entry to the measurements list
                for measurement in measurements[visit_id][person_id].values():
                    loop_visit['measurements'].append(measurement)
                
                #//*** Add the Completed loop_visit entry to the person_id visits list
                people[person_id]["visits"].append(loop_visit)
        
        #//*** Dictionary Created, Load dictionary into TinyDB
        self._load_db(people)

        
        
    def _load_db(self,input_json):
        
        self._db = TinyDB(self._db_path)
        
        for key,value in input_json.items():
            self._db.insert({'name': key, 'data' : value})
        

    #//*** Load JSON from file as dictionary
    def _read_file(self,jsonpath):
        if jsonpath.exists():
            with open(jsonpath) as f:
                return json.load(f)
    
    def close(self):
        #//*** Close the Database
        self._db.close()
        

#//**************************
#//*** END class DocumentDB
#//**************************

#//*** Path to store the TinyDB JSON file
db_path = results_dir.joinpath('patient-info.json')

#//*** Delete any existing Databases. Wrapping this in a try because sometimes 
#//*** things go wrong
try:
    if db_path.exists():
        os.remove(db_path)
except:
    print("Can't Close")

#//*** Initialize the TinyDB
db = DocumentDB(db_path)

#//*** Close the db.
db.close()